# Building Abstractions with Data

In this chapter we are going to look at more complex data. Thus we turn to another key aspect of any programming language: the means it provides for building abstractions by combining data objects to form *compound data*.

Why do we want compound data? The same reason we want compound functions: to elevate the conceptual level at which we can design our programs, to increase the modulatiry of our designs, and to enhance the expressive power of our language. The ability to construct compound data enables us to deal with data at a higher conceptual level than that of the primitive data objects of the language.

Consider a program that performs arithmetic with rational numbers. We can imagine an operation `add_rat` that takes two rational numbers and produces their sum. In terms of primitive data, a rational number can be thought of as two integers: a numerator and a denominator. Thus, we could design a program in which each rational number would be represented by two integers (a numerator and a denominator) and where `add_rat` would be implemented by two functions (one producing the numerator of the sum and one producing the denominator). But this would be awkward, because we would then need to explicitly keep track of which numerators corresponded to which denominators. In a system intended to perform many operations on many rational numbers, such bookkeeping details would clutter the programs substantially, to say nothing of what they would do to our minds. It would be much better if we could “glue together” a numerator and denominator to form a pair--a *compound data object*--that our programs could manipulate in a way that would be consistent with regarding a rational number as a single conceptual unit.

Use of compound data also enables us to increase the modularity of our programs. If we can manipulate rational numbers directly as objects in their own right, then we can separate the part of our program that deals with rational numbers per se from the details of how rational numbers may be represented as pairs of integers. The general technique of isolating the parts of a program that deal with how data objects are represented from the parts of a program that deal with how data objects are used is a powerful design methodology called *data abstraction*. We will see how data abstraction makes programs much easier to design, maintain, and modify.

The use of compound data also leads to a real increase in the expressive power of our programming language. Consider the idea of forming a "linear combination", $ax + by$. We might like to write a function that would accept $a$, $b$, $x$, and $$ as arguments and return the value of $ax + by$. This presents no difficulty if the argumetns are to be numbers, because we can readily declare the function

```javascript
function linear_combination(a, b, x, y) {
    return a * x + b * y;
}
```

But suppose we are not concerned only with numbers. Suppose we would like to describe a process that forms linear combinations whenever addition and multiplication are defined—for rational numbers, complex numbers, polynomials, or whatever. We could express this as a function of the form

```javascript
function linear_combination(a, b, x, y) {
    return add(mul(a, x), mul(b, y));
}
```

where `add` and `mul` are not the primitive functions `+` and `*` but rather more complex things that will perform the appropriate operations for whatever kinds of data we pass in as the arguments `a`, `b`, `x`, and `y`. The key point is that the only thing `linear_combination` should need to know about `a`, `b`, `x`, and `y` is that the functions `add` and `mul` will perform the appropriate manipulations. From the perspective of the function `linear_combination`, it is irrelevant what `a`, `b`, `x`, and `y` are and even more irrelevant how they might happen to be represented in terms of more primitive data. This same example shows why it is important that our programming language provide the ability to manipulate compound objects directly: Without this, there is no way for a function such as `linear_combination` to pass its arguments along to `add` and `mul` without having to know their detailed structure.

We begin this chapter by implementing the rational-number arithmetic system described above. The main issue to be addressed is that of abstraction as a technique for coping with complexity, and we will see how data abstraction enables us to erect suitable *abstraction barriers* between different parts of a program.

We will see that the key to forming compound data is that a programming language should provide some kind of “glue” so that data objects can be combined to form more complex data objects. There are many possible kinds of glue. Indeed, we will discover how to form compound data using no special “data” operations at all, only functions. This will further blur the distinction between “function” and “data,” which was already becoming tenuous toward the end of chapter 1. One key idea in dealing with compound data is the notion of *closure*—that the glue we use for combining data objects should allow us to combine not only primitive data objects, but compound data objects as well. Another key idea is that compound data objects can serve as *conventional interfaces* for combining program modules in mix-and-match ways. We illustrate some of these ideas by presenting a simple graphics language that exploits closure.

We will then augment the representational power of our language by introducing *symbolic expressions*—data whose elementary parts can be arbitrary symbols rather than only numbers.

Next we will take up the problem of working with data that may be represented differently by different parts of a program. This leads to the need to implement *generic operations*, which must handle many different types of data. Maintaining modularity in the presence of generic operations requires more powerful abstraction barriers than can be erected with simple data abstraction alone. In particular, we introduce *data-directed programming* as a technique that allows individual data representations to be designed in isolation and then combined *additively* (i.e., without modification).

## 2.1 Introduction to Data Abstraction

In section 1.1.8, we noted that a function used as an element in creating a more complex function could be regarded not only as a collection of particular operations but also as a functional abstraction. The analogous notion for compound data is called *data abstraction*. Data abstraction is a methodology that enables us to isolate how a compound data object is used from the details of how it is constructed from more primitive data objects.

The basic idea of data abstraction is to structure the programs that are to use compound data objects so that they operate on “abstract data.” That is, our programs should use data in such a way as to make no assumptions about the data that are not strictly necessary for performing the task at hand. At the same time, a “concrete” data representation is defined independent of the programs that use the data. The interface between these two parts of our system will be a set of functions, called *selectors* and *constructors*, that implement the abstract data in terms of the concrete representation. To illustrate this technique, we will consider how to design a set of functions for manipulating rational numbers.

### 2.1.1 Example: Arithmetic Operations for Rational Numbers

Suppose we want to do arithmetic with rational numbers. We want to be able to add, subtract, multiply, and divide them and to test whether two rational numbers are equal.

Let us begin by assuming that we already have a way of constructing a rational number from a numerator and a denominator. We also assume that, given a rational number, we have a way of extracting (or selecting) its numerator and its denominator. Let us further assume that the constructor and selectors are available as functions:

- `make_rat(n, d)` returns the rational number whose numerator is the integer $n$ and whose denominator is the integer $d$.
- `numer(x)` returns the numerator of the rational number $x$.
- `denom(x)` returns the denominator of the rational number $x$.

We are using here a powerful strategy of synthesis: *wishful thinking*. We haven’t yet said how a rational number is represented, or how the functions `numer`, `denom`, and `make_rat` should be implemented. Even so, if we did have these three functions, we could then add, subtract, multiply, divide, and test equality by using the following relations:

$$
\begin{align}
\frac{n_1}{d_1} + \frac{n_2}{d_2} &= \frac{n_1 d_2 + n_2 d_1}{d_1 d_2} \\
\frac{n_1}{d_1} - \frac{n_2}{d_2} &= \frac{n_1 d_2 - n_2 d_1}{d_1 d_2} \\
\frac{n_1}{d_1} \cdot \frac{n_2}{d_2} &= \frac{n_1 n_2}{d_1 d_2} \\
\frac{n_1 / d_1}{n_2 / d_2} &= \frac{n_1 d_2}{d_1 n_2} \\
\frac{n_1}{d_1} &= \frac{n_2}{d_2} \text{ if and only if } n_1 d_2 = n_2 d_1
\end{align}
$$

We can express these rules as functions:

In [40]:
function add_rat(x, y) {
    return make_rat(numer(x) * denom(y) + numer(y) * denom(x), denom(x) * denom(y));
}

In [41]:
function sub_rat(x, y) {
    return make_rat(numer(x) * denom(y) - numer(y) * denom(x), denom(x) * denom(y)); 
}

In [42]:
function mul_rat(x, y) {
    return make_rat(numer(x) * numer(y), denom(x) * denom(y)); 
}


In [43]:
function div_rat(x, y) {
    return make_rat(numer(x) * denom(y), denom(x) * numer(y)); 
} 

In [44]:
function equal_rat(x, y) {
    return numer(x) * denom(y) === numer(y) * denom(x);
}

Now we have the operations on rational numbers defined in terms of the selector and constructor functions `numer`, `denom`, and `make_rat`. But we haven’t yet defined these. What we need is some way to glue together a numerator and a denominator to form a rational number.

#### Pairs

To enable us to implement the concrete level of our data abstraction, we define a compound structure called a *pair*, which can be constructed with a function `pair`. This function takes two arguments and returns a compound data object (an array) that contains the two arguments as parts.

In [2]:
function pair(x, xs){
  return [x, xs]
}

Given a pair, we can extract the parts using the primitive functions `head` and `tail`.

In [4]:
function array_test(x) {
  if (Array.isArray === undefined) {
    return x instanceof Array
  } else {
    return Array.isArray(x)
  }
}

In [3]:
// is_pair returns true iff arg is a two-element array
function is_pair(x){
  return array_test(x) && x.length === 2
}

In [9]:
// head returns the first component of the given pair,
// throws an exception if the argument is not a pair
function head(xs) {
  if (is_pair(xs)) {
    return xs[0]
  } else {
    throw new Error('head(xs) expects a pair as argument xs, but encountered ' + stringify(xs))
  }
}

In [ ]:
// tail returns the second component of the given pair
// throws an exception if the argument is not a pair
function tail(xs) {
  if (is_pair(xs)) {
    return xs[1]
  } else {
    throw new Error('tail(xs) expects a pair as argument xs, but encountered ' + stringify(xs))
  }
}

Thus, we can use pair, head, and tail as follows:

In [12]:
const x = pair(1, 2);

In [13]:
head(x);

1

In [14]:
tail(x);

2

Notice that a pair is a data object that can be given a name and manipulated, just like a primitive data object. Moreover, `pair` can be used to form pairs whose elements are pairs, and so on:

In [15]:
const x = pair(1, 2);

In [16]:
const y = pair(3, 4);

In [17]:
const z = pair(x, y);

In [18]:
head(head(z));

1

In [19]:
head(tail(z));

3

In section 2.2 we will see how this ability to combine pairs means that pairs can be used as general-purpose building blocks to create all sorts of complex data structures. The single compound-data primitive *pair*, implemented by the functions `pair`, `head`, and `tail`, is the only glue we need. Data objects constructed from pairs are called *list-structured* data.

#### Representing rational numbers

Pairs offer a natural way to complete the rational-number system. Simply represent a rational number as a pair of two integers: a numerator and a denominator. Then `make_rat`, `numer`, and `denom` are readily implemented as follows:

In [20]:
function make_rat(n, d) { return pair(n, d); }
function numer(x) { return head(x); }
function denom(x) { return tail(x); }

Another way to define the selectors and constructor would be

```javascript
const make_rat = pair;
const numer = head;
const denom = tail;
````
    
The first definition associates the name `make_rat` with the value of the expression `pair`, which is the primitive function that constructs pairs. Thus `make_rat` and `pair` are names for the same primitive constructor.

Defining selectors and constructors in this way is efficient: Instead of `make_rat` calling `pair`, `make_rat` *is* `pair`, so there is only one function called, not two, when `make_rat` is called. On the other hand, doing this defeats debugging aids that trace function calls or put breakpoints on function calls: You may want to watch `make_rat` being called, but you certainly don’t want to watch every call to `pair`.

We have chosen not to use this style of definition in this book.

Also, in order to display the results of our computations, we can print rational numbers by printing the numerator, a slash, and the denominator. We can use the primitive function `stringify` to turn any value (here a number) into a string. To combine the numbers that we `stringify`, we can use the operator `+` in JavaScript, that is *overloaded*; it can be applied to two numbers or to two strings, and in the latter case it returns the result of *concatenating* the two strings.

In [34]:
function print_rat(x) {
    return JSON.stringify(numer(x)) + " / " + JSON.stringify(denom(x));
}

How we can try our rational-number functions:

In [36]:
const one_half = make_rat(1, 2);

In [37]:
print_rat(one_half);

"1 / 2"

In [38]:
const one_third = make_rat(1, 3);

In [45]:
print_rat(add_rat(one_half, one_third));

"5 / 6"

In [47]:
print_rat(mul_rat(one_half, one_third));

"1 / 6"

In [48]:
print_rat(add_rat(one_third, one_third));

"6 / 9"

As the final example shows, our rational-number implementation does not reduce rational numbers to lowest terms. We can remedy this by changing `make_rat`. If we have a `gcd` function like the one in section 1.2.5 that produces the greatest common divisor of two integers, we can use `gcd` to reduce the numerator and the denominator to lowest terms before constructing the pair:

In [49]:
function gcd(a, b) {
    return b === 0 ? a : gcd(b, a % b); 
}

In [50]:
function make_rat(n, d) {
    const g = gcd(n, d);
    return pair(n / g, d / g);
}

Now we have

In [51]:
print_rat(add_rat(one_third, one_third));

"2 / 3"

as desired. This modification was accomplished by changing the constructor `make_rat` without changing any of the functions (such as `add_rat` and `mul_rat`) that implement the actual operations.

#### Exercise 2.1

Define a better version of `make_rat` that handles both positive and negative arguments. The function `make_rat` should normalize the sign so that if the rational number is positive, both the numerator and denominator are positive, and if the rational number is negative, only the numerator is negative.

In [ ]:
function make_rat(n, d) {
    const g = gcd(n, d);
    return pair(n / g, d / g);
}

In [52]:
gcd(-2, -4);

-2

In [53]:
-2 % -4

-2

In [57]:
-2 / -4

0.5

### 2.1.2 Abstraction Barriers

Before continuing with more examples of compound data and data abstraction, let us consider some of the issues raised by the rational-number example. We defined the rational-number operations in terms of a constructor `make_rat` and selectors `numer` and `denom`. **In general, the underlying idea of data abstraction is to identify for each type of data object a basic set of operations in terms of which all manipulations of data objects of that type will be expressed, and then to use only those operations in manipulating the data.**

![fig 2.1](./figs/fig2.1.png)

We can envision the structure of the rational-number system as shown in figure 2.1. **The horizontal lines represent *abstraction barriers* that isolate different “levels” of the system. At each level, the barrier separates the programs (above) that use the data abstraction from the programs (below) that implement the data abstraction.** Programs that use rational numbers manipulate them solely in terms of the functions supplied “for public use” by the rational-number package: `add_rat`, `sub_rat`, `mul_rat`, `div_rat`, and `equal_rat`. These, in turn, are implemented solely in terms of the constructor and selectors `make_rat`, `numer`, and `denom`, which themselves are implemented in terms of pairs. The details of how pairs are implemented are irrelevant to the rest of the rational-number package so long as pairs can be manipulated by the use of `pair`, `head`, and `tail`. **In effect, functions at each level are the interfaces that define the abstraction barriers and connect the different levels.**

This simple idea has many advantages. One advantage is that it makes programs much easier to maintain and to modify. Any complex data structure can be represented in a variety of ways with the primitive data structures provided by a programming language. Of course, the choice of representation influences the programs that operate on it; thus, if the representation were to be changed at some later time, all such programs might have to be modified accordingly. This task could be time-consuming and expensive in the case of large programs unless the dependence on the representation were to be confined by design to a very few program modules.

For example, an alternate way to address the problem of reducing rational numbers to lowest terms is to perform the reduction whenever we access the parts of a rational number, rather than when we construct it. This leads to different constructor and selector functions:

```javascript
function make_rat(n, d) {
    return pair(n, d);
}
function numer(x) {
    const g = gcd(head(x), tail(x));
    return head(x) / g;
}
function denom(x) {
    const g = gcd(head(x), tail(x));
    return tail(x) / g;
}
```

The difference between this implementation and the previous one lies in when we compute the `gcd`. If in our typical use of rational numbers we access the numerators and denominators of the same rational numbers many times, it would be preferable to compute the `gcd` when the rational numbers are constructed. If not, we may be better off waiting until access time to compute the `gcd`. In any case, when we change from one representation to the other, the functions `add_rat`, `sub_rat`, and so on do not have to be modified at all.

#### Exercise 2.2

Consider the problem of representing line segments in a plane. Each segment is represented as a pair of points: a starting point and an ending point. Declare a constructor `make_segment` and selectors `start_segment` and `end_segment` that define the representation of segments in terms of points. Furthermore, a point can be represented as a pair of numbers: the $x$ coordinate and the $y$ coordinate. Accordingly, specify a constructor `make_point` and selectors `x_point` and `y_point` that define this representation. Finally, using your selectors and constructors, declare a function `midpoint_segment` that takes a line segment as argument and returns its midpoint (the point whose coordinates are the average of the coordinates of the endpoints). To try your functions, you’ll need a way to print points:

In [58]:
function make_point(x, y) {
    return pair(x, y);
}

In [59]:
function x_point(point) {
    return head(point);
}

In [60]:
function y_point(point) {
    return tail(point);
}

In [61]:
function make_segment(start_point, end_point) {
    return pair(start_point, end_point);
}

In [62]:
function start_segment(segment) {
    return head(segment);
}

In [63]:
function end_segment(segment) {
    return tail(segment);
}

In [ ]:
function print_point(p) {
    return display("(" + JSON.stringify(x_point(p)) + ", " + JSON.stringify(y_point(p)) + ")"); 
}

In [ ]:
function midpoint_segment(segment) {
    return point(
        (x_point(start_segment(segment)) + x_point(end_segment(segment))) / 2
        (y_point(start_segment(segment)) + y_point(end_segment(segment))) / 2
    );
}